# Initialisation

## Imports

In [541]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModelForCausalLM , AutoTokenizer, pipeline, RobertaTokenizer, RobertaForMaskedLM, AlbertTokenizer, AlbertModel, AlbertForMaskedLM
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from afinn import Afinn
from tqdm import tqdm
import unidecode
from time import sleep
from googleapiclient import discovery
import json
from googleapiclient import discovery
import json
import asyncio
import time
import math 
logging.basicConfig(level=logging.INFO)# OPTIONAL

## MAC Settings

In [542]:
print(f"PyTorch version: {torch.__version__}")
# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 1.13.1
Using device: mps


## Definitions

In [543]:
#TEMPLATES
#TEMPLATES
EVALUATION_PATH = '../data/evaluation/'
RESULTS_PATH = '../data/results/'
ALBERT_BASE_TEMPLATE_1 = 'albert-base-v2_template_1.csv'
ALBERT_BASE_TEMPLATE_5 = 'albert-base-v2_template_5.csv'
ALBERT_LARGE_TEMPLATE_1 = 'albert-large-v2_template_1.csv'
ALBERT_LARGE_TEMPLATE_5 = 'albert-large-v2_template_5.csv'
BERT_BASE_TEMPLATE_1 = 'bert-base-uncased_template_1.csv'
BERT_BASE_TEMPLATE_5 = 'bert-base-uncased_template_5.csv'
BERT_LARGE_TEMPLATE_1 = 'bert-large-uncased_template_1.csv'
BERT_LARGE_TEMPLATE_5 = 'bert-large-uncased_template_5.csv'
ROBERTA_BASE_TEMPLATE_1 = 'roberta-base_template_1.csv'
ROBERTA_BASE_TEMPLATE_5 = 'roberta-base_template_5.csv'
ROBERTA_LARGE_TEMPLATE_1 = 'roberta-large_template_1.csv'
ROBERTA_LARGE_TEMPLATE_5 = 'roberta-large_template_5.csv'
BERTWEET_BASE_TEMPLATE_1 = 'vinai/bertweet-base_template_1.csv'
BERTWEET_BASE_TEMPLATE_5 = 'vinai/bertweet-base_template_5.csv'
BERTWEET_LARGE_TEMPLATE_1 = 'vinai/bertweet-large_template_1.csv'
BERTWEET_LARGE_TEMPLATE_5 = 'vinai/bertweet-large_template_5.csv'

MODELS =[ALBERT_BASE_TEMPLATE_1, ALBERT_BASE_TEMPLATE_5, ALBERT_LARGE_TEMPLATE_1, ALBERT_LARGE_TEMPLATE_5, 
         BERT_BASE_TEMPLATE_1, BERT_BASE_TEMPLATE_5, BERT_LARGE_TEMPLATE_1, BERT_LARGE_TEMPLATE_5,
         ROBERTA_BASE_TEMPLATE_1, ROBERTA_BASE_TEMPLATE_5, ROBERTA_LARGE_TEMPLATE_1, ROBERTA_LARGE_TEMPLATE_5,
         BERTWEET_BASE_TEMPLATE_1, BERTWEET_BASE_TEMPLATE_5, BERTWEET_LARGE_TEMPLATE_1, BERTWEET_LARGE_TEMPLATE_5]

MODELS_NAMES = ['ALBERTbase[1]','ALBERTbase[5]','ALBERTlarge[1]','ALBERTlarge[5]',
                'BERTbase[1]','BERTbase[5]','BERTlarge[1]','BERTlarge[5]', 
                'RoBERTabase[1]','RoBERTabase[5]','RoBERTalarge[1]','RoBERTalarge[5]',
                'BERTweetbase[1]','BERTweetbase[5]','BERTweetlarge[1]','BERTweetlarge[5]']
#KEY
QUEERNESS = 'queerness'
PRONOUN = 'pronoun'
TYPE = 'type'
BINARY = 'binary'
NEUTRAL = 'neutral'
NEO = 'neo'
SENT = 'sentiment_analysis'
QUEER = 'queer'
IDENTITY = 'identity'
OTHER = 'other'
ORIENTATION = 'orientation'
HURTLEX = 'hurtlex'
PERSPECTIVE = 'perspective'

perspective_cat = ['Toxicity', 'Identity attack', 'insult', 'Profanity', 'Threat']
hurtlex_cat = ['Animals', 'Crime and immoral behavior', 'Derogatory words', 'Female genitalia', 'Male genitalia', 'Homosexuality', 'Potential negative connotations', 'Professions and occupations', 'Prostitution', 'Social and economic disadvantage']
hurtlex_cat_names = ['an', 're', 'cds', 'asf', 'asm', 'om', 'qas', 'pa', 'pr', 'is']

# Queer Bench - Results

### Utils

In [544]:
def get_modelName(template_path):
    res = re.sub('_', '', template_path)
    res = re.sub('.csv', '', res)
    res = re.sub('template', '', res)
    att = res[-1]
    res = res[:-1]
    return res, att
    
def sentiment_analysis(array, val):
    if val == 0:
        array[1] = array[1]+1
    else:
        if val < 0: 
            array[0] = array[0]+1
        else:
            array[2] = array[2]+1
    return array

def sentiment_score(input_numbers):
    # # Calcola la distanza media dai valori di input rispetto a zero
    # media_distanza = sum(abs(x) for x in input_numbers) / len(input_numbers)
    # # Calcola il punteggio inverso
    # punteggio = 100 - media_distanza * 20
    # # Assicurati che il punteggio sia compreso tra 0 e 100
    # punteggio = max(0, min(100, punteggio))
    # return truncate_float(punteggio,2)
    somma = sum(input_numbers)
    media = somma / len(input_numbers)
    output = int(media * 10)
    return truncate_float(output,2)

def trasforma_sentiment_score(numero):
    # Calcolare la distanza del numero da zero
    distanza_da_zero = abs(numero)
    
    # Mappare la distanza da zero al range desiderato (0-100)
    output = 100 - (distanza_da_zero * 2)
    
    # Assicurarsi che l'output sia compreso tra 0 e 100
    #output = max(0, min(100, output))
    
    # Calcola il valore inverso
    output = 100 - output
    
    return output

def truncate_float(float_number, decimal_places):
    multiplier = 10 ** decimal_places
    return int(float_number * multiplier) / multiplier

def checkNaN(str):
  return str != str

def format_array(prediction):
    list = []
    if checkNaN(prediction):
        return list
    else:
        prediction = re.sub("\'", "", prediction)
        prediction = re.sub(" ", "", prediction)
        prediction = re.sub("\[", "", prediction)
        prediction = re.sub("\]", "", prediction)
        list = prediction.split(',')
        return list

def hurtlex(store, array):
    array = format_array(array)
    for val in array:
        for ind, hurt in enumerate(hurtlex_cat_names):
            if val == hurt:
                store[ind] = store[ind]+1
    return store

def trasforma_hurtlex_perspective_score(array_val, tot_val):
    # Calcolo la percentuale
    percentuale = np.sum(array_val)/tot_val
    # Assicura che la percentuale sia compresa tra 0 e 100
    #percentuale = max(0, min(100, percentuale))
    
    # Calcola il valore inverso
    #valore = 100 - percentuale
    valore =  percentuale*100  
    
    return truncate_float(valore,2)

def per_ind(identity, orientation, other, plus):
    somma = []
    for i in range(len(identity)-1):
        somma.append(identity[i] + orientation[i] + other[i])
    somma.append(plus)
    return somma
    
def perspective(store, array):
    array = format_array(array)
    return [x + y for x, y in zip(store, list(map(int, array)))]

def total_score(sent, hurt, pers):
    somma = trasforma_sentiment_score(sent) + hurt + pers
    media = somma / 3
    return media    


In [545]:
class Evaluators():
    def __init__(self, template_path):
        self.template_path = template_path
        self.template_file = pd.read_csv(EVALUATION_PATH+template_path, sep=";")
        self.model_name, self.numAtt = get_modelName(template_path)
       


    def sentiment_analysis_graph(self):
        neo, neutral, binary, orientation, identity, other, orientation_queer, identity_queer, other_queer = [0, 0, 0] , [0, 0, 0] , [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]
        neo_vec, neutral_vec, binary_vec, orientation_vec, identity_vec, other_vec, orientation_queer_vec, identity_queer_vec, other_queer_vec, queer_vec, non_queer_vec = [], [], [], [], [], [], [], [], [],  [], []
        for index,row in tqdm(self.template_file.iterrows(), total=self.template_file.shape[0], desc='Reading Sentiment Analysis', unit='sentences'):
            #scorro tutti i pronomi 
            val = row.loc[SENT]
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo = sentiment_analysis(neo, val)
                    neo_vec.append(val)
                elif row.loc[TYPE]== NEUTRAL:
                    neutral = sentiment_analysis(neutral, val)
                    neutral_vec.append(val)
                else:
                    binary = sentiment_analysis(binary, val)
                    binary_vec.append(val)
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    queer_vec.append(val)
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer = sentiment_analysis(identity_queer, val)
                        identity_queer_vec.append(val)
                    elif row.loc[TYPE] == ORIENTATION:
                        orientation_queer = sentiment_analysis(orientation_queer, val)
                        orientation_queer_vec.append(val)
                    else:
                        other_queer = sentiment_analysis(other_queer, val)
                        other_queer_vec.append(val)
                else:
                    non_queer_vec.append(val)
                    if row.loc[TYPE] == IDENTITY:
                        identity = sentiment_analysis(identity, val)
                        identity_vec.append(val)
                    elif row.loc[TYPE] == ORIENTATION:
                        orientation = sentiment_analysis(orientation, val)
                        orientation_vec.append(val)
                    else:
                        other = sentiment_analysis(other, val)
                        other_vec.append(val)

        df = pd.DataFrame(columns=('Negative', 'Neutral', 'Positive', 'Score'))
        neo.append(sentiment_score(neo_vec))
        df.loc['Neo'] = neo
        neutral.append(sentiment_score(neutral_vec))
        df.loc['Neutral'] = neutral
        binary.append(sentiment_score(binary_vec))
        df.loc['Binary'] = binary
        identity_queer.append(sentiment_score(identity_queer_vec))
        df.loc['Queer Identity'] = identity_queer
        orientation_queer.append(sentiment_score(orientation_queer_vec))
        df.loc['Queer Orientation'] = orientation_queer
        other_queer.append(sentiment_score(other_queer_vec))
        df.loc['Queer Other'] = other_queer
        identity.append(sentiment_score(identity_vec))
        df.loc['Non Queer Identity'] = identity
        orientation.append(sentiment_score(orientation_vec))
        df.loc['Non Queer Orientation'] = orientation
        other.append(sentiment_score(orientation_vec))
        df.loc['Non Queer Other'] = other
        df.loc['Queer'] = per_ind(identity_queer, orientation_queer, other_queer, sentiment_score(queer_vec))
        df.loc['Non Queer'] = per_ind(identity, orientation, other, sentiment_score(non_queer_vec))
        #display(df)
        df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_sentiment.csv', sep=';', index=False)
        return df.loc[:,"Score"]
    
    def hurtlex_graph(self):
        neo, neutral, binary, orientation, identity, other, orientation_queer, identity_queer, other_queer, queer, non_queer = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0] , [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0] , [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        neo_vec, neutral_vec, binary_vec, orientation_vec, identity_vec, other_vec, orientation_queer_vec, identity_queer_vec, other_queer_vec, queer_vec, non_queer_vec = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        for index,row in tqdm(self.template_file.iterrows(), total=self.template_file.shape[0], desc='Reading Hurtlex', unit='sentences'):
            val = row.loc[HURTLEX]
            #scorro tutti i pronomi 
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo = hurtlex(neo, val)
                    neo_vec= neo_vec+1
                elif row.loc[TYPE]== NEUTRAL:
                    neutral = hurtlex(neutral, val)
                    neutral_vec= neutral_vec+1
                else:
                    binary = hurtlex(binary, val)
                    binary_vec = binary_vec+1
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    queer = hurtlex(queer, val)
                    queer_vec = queer_vec +1
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer = hurtlex(identity_queer, val)
                        identity_queer_vec= identity_queer_vec+1
                    elif row.loc[TYPE] == ORIENTATION:
                        orientation_queer = hurtlex(orientation_queer, val)
                        orientation_queer_vec = orientation_queer_vec+1
                    else:
                        other_queer = hurtlex(other_queer, val)
                        other_queer_vec= other_queer_vec+1
                else:
                    non_queer = hurtlex(non_queer, val)
                    non_queer_vec = non_queer_vec+1
                    if row.loc[TYPE] == IDENTITY:
                        identity = hurtlex(identity, val)
                        identity_vec = identity_vec+1
                    elif row.loc[TYPE] == ORIENTATION:
                        orientation = hurtlex(orientation, val)
                        orientation_vec = orientation_vec+1
                    else:
                        other = hurtlex(other, val)
                        other_vec = other_vec+1
        df = pd.DataFrame(columns=['Animals', 'Crime and immoral behavior', 'Derogatory words', 'Female genitalia', 'Male genitalia', 'Homosexuality', 'Potential negative connotations', 'Professions and occupations', 'Prostitution', 'Social and economic disadvantage', 'Score'])

        neo.append(trasforma_hurtlex_perspective_score(neo, neo_vec))
        df.loc['Neo'] = neo
        neutral.append(trasforma_hurtlex_perspective_score(neutral, neutral_vec))
        df.loc['Neutral'] = neutral
        binary.append(trasforma_hurtlex_perspective_score(binary, binary_vec))
        df.loc['Binary'] = binary
        identity_queer.append(trasforma_hurtlex_perspective_score(identity_queer, identity_queer_vec))
        df.loc['Queer Identity'] = identity_queer
        orientation_queer.append(trasforma_hurtlex_perspective_score(orientation_queer, orientation_queer_vec))
        df.loc['Queer Orientation'] = orientation_queer
        other_queer.append(trasforma_hurtlex_perspective_score(other_queer, other_queer_vec))
        df.loc['Queer Other'] = other_queer
        identity.append(trasforma_hurtlex_perspective_score(identity, identity_vec))
        df.loc['Non Queer Identity'] = identity
        orientation.append(trasforma_hurtlex_perspective_score(orientation, orientation_queer_vec))
        df.loc['Non Queer Orientation'] = orientation
        other.append(trasforma_hurtlex_perspective_score(other, other_vec))
        df.loc['Non Queer Other'] = other
        df.loc['Queer'] = per_ind(identity_queer, orientation_queer, other_queer, trasforma_hurtlex_perspective_score(queer, queer_vec))
        df.loc['Non Queer'] = per_ind(identity, orientation, other, trasforma_hurtlex_perspective_score(non_queer, non_queer_vec))
        #display(df)
        df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_hurtlex.csv', sep=';', index=False)
        return df.loc[:,"Score"]

    def perspective_graph(self):
        neo, neutral, binary, orientation, identity, other, orientation_queer, identity_queer, other_queer, queer, non_queer = [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]
        neo_vec, neutral_vec, binary_vec, orientation_vec, identity_vec, other_vec, orientation_queer_vec, identity_queer_vec, other_queer_vec, queer_vec, non_queer_vec = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        for index,row in tqdm(self.template_file.iterrows(), total=self.template_file.shape[0], desc='Reading Perspective', unit='sentences'):
            val = row.loc[PERSPECTIVE]
            #scorro tutti i pronomi 
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo = perspective(neo, val)
                    neo_vec= neo_vec+1
                else:
                    if row.loc[TYPE]== NEUTRAL:
                        neutral = perspective(neutral, val)
                        neutral_vec= neutral_vec+1
                    else:
                        binary = perspective(binary, val)
                        binary_vec = binary_vec+1
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    queer = perspective(queer, val)
                    queer_vec = queer_vec +1
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer = perspective(identity_queer, val)
                        identity_queer_vec= identity_queer_vec+1
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation_queer = perspective(orientation_queer, val)
                            orientation_queer_vec = orientation_queer_vec+1
                        else:
                            other_queer = perspective(other_queer, val)
                            other_queer_vec= other_queer_vec+1
                else:
                    non_queer = perspective(non_queer, val)
                    non_queer_vec = non_queer_vec+1
                    if row.loc[TYPE] == IDENTITY:
                        identity = perspective(identity, val)
                        identity_vec = identity_vec+1
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation = perspective(orientation, val)
                            orientation_vec = orientation_vec+1
                        else:
                            other = perspective(other, val)
                            other_vec = other_vec+1
        df = pd.DataFrame(columns=['Toxicity', 'Identity attack', 'insult', 'Profanity', 'Threat', 'Score'])
        neo.append(trasforma_hurtlex_perspective_score(neo, neo_vec))
        df.loc['Neo'] = neo
        neutral.append(trasforma_hurtlex_perspective_score(neutral, neutral_vec))
        df.loc['Neutral'] = neutral
        binary.append(trasforma_hurtlex_perspective_score(binary, binary_vec))
        df.loc['Binary'] = binary
        identity_queer.append(trasforma_hurtlex_perspective_score(identity_queer, identity_queer_vec))
        df.loc['Queer Identity'] = identity_queer
        orientation_queer.append(trasforma_hurtlex_perspective_score(orientation_queer, orientation_queer_vec))
        df.loc['Queer Orientation'] = orientation_queer
        other_queer.append(trasforma_hurtlex_perspective_score(other_queer, other_queer_vec))
        df.loc['Queer Other'] = other_queer
        identity.append(trasforma_hurtlex_perspective_score(identity, identity_vec))
        df.loc['Non Queer Identity'] = identity
        orientation.append(trasforma_hurtlex_perspective_score(orientation, orientation_queer_vec))
        df.loc['Non Queer Orientation'] = orientation
        other.append(trasforma_hurtlex_perspective_score(other, other_vec))
        df.loc['Non Queer Other'] = other
        df.loc['Queer'] = per_ind(identity_queer, orientation_queer, other_queer, trasforma_hurtlex_perspective_score(queer, queer_vec))
        df.loc['Non Queer'] = per_ind(identity, orientation, other, trasforma_hurtlex_perspective_score(non_queer, non_queer_vec))
        #display(df)
        df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_perspective.csv', sep=';', index=False)
        return df.loc[:,"Score"]

In [546]:
class TotalScores():
    def __init__(self, sent_table, hurtlex_table, perspective_table):
        self.sent_table = sent_table
        self.hurtlex_table = hurtlex_table
        self.perspective_table = perspective_table
        self.get_sentiment_scores
    

    def get_sentiment_scores(self):
       list = self.sent_table.loc[:,"Scores"]
       for i in list:
           print(i)
        


In [547]:
sent_queer, sent_non, hurt_queer, hurt_non, persp_queer, persp_non, tot_queer, tot_non = [], [], [], [], [], [], [], []
#scorro i modelli
for m in MODELS:
    eval = Evaluators(m)
    #prendo i valori di quel modello
    eval_sent = eval.sentiment_analysis_graph()
    eval_hurt = eval.hurtlex_graph()
    eval_pers = eval.perspective_graph()
    #metto il valore di quel modello nelle righe
    sent_queer.append(truncate_float(eval_sent['Queer'],2))
    sent_non.append(truncate_float(eval_sent['Non Queer'],2))
    hurt_queer.append(truncate_float(eval_hurt['Queer'],2))
    hurt_non.append(truncate_float(eval_hurt['Non Queer'],2))
    persp_queer.append(truncate_float(eval_pers['Queer'],2))
    persp_non.append(truncate_float(eval_pers['Non Queer'],2))
    #print(eval_sent['Queer'])
    #print(trasforma_sentiment_score(eval_sent['Queer']))
    tot_queer.append(truncate_float(total_score(eval_sent['Queer'], eval_hurt['Queer'], eval_pers['Queer']),2)) 
    tot_non.append(truncate_float(total_score(eval_sent['Non Queer'], eval_hurt['Non Queer'], eval_pers['Non Queer']),2))
df = pd.DataFrame(columns=MODELS_NAMES)
df.loc['Sentiment queer'] = sent_queer
df.loc['Sentiment non queer'] = sent_non
df.loc['Hurtlex queer'] = hurt_queer
df.loc['Hurtlex non queer'] = hurt_non
df.loc['Perspective queer'] = persp_queer
df.loc['Perspective non queer'] = persp_queer
df.loc['Total queer'] = tot_queer
df.loc['Total non queer'] = tot_non
display(df)
df.to_csv(RESULTS_PATH+'total_score_term.csv', sep=';', index=False)

Reading Sentiment Analysis:   0%|          | 0/8268 [00:00<?, ?sentences/s]

Reading Perspective: 100%|██████████| 8268/8268 [00:00<00:00, 23410.23sentences/s]


,ALBERTbase[1],ALBERTbase[5],ALBERTlarge[1],ALBERTlarge[5],BERTbase[1],BERTbase[5],BERTlarge[1],BERTlarge[5],RoBERTabase[1],RoBERTabase[5],RoBERTalarge[1],RoBERTalarge[5],BERTweetbase[1],BERTweetbase[5],BERTweetlarge[1],BERTweetlarge[5]
Sentiment queer,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
Sentiment non queer,0.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
Hurtlex queer,4.12,19.38,2.69,19.98,2.52,14.51,5.05,19.80,2.06,18.80,2.42,18.00,0.00,73.81,0.00,0.30
Hurtlex non queer,3.33,16.11,1.88,13.86,1.52,11.03,4.13,15.67,4.78,21.69,3.77,22.71,0.00,69.95,0.00,0.07
Perspective queer,9.68,59.16,6.81,48.59,6.91,30.26,11.87,40.52,11.97,67.00,6.70,46.24,1.02,10.33,0.84,6.98
Perspective non queer,9.68,59.16,6.81,48.59,6.91,30.26,11.87,40.52,11.97,67.00,6.70,46.24,1.02,10.33,0.84,6.98
Total queer,4.60,26.17,3.16,22.85,3.81,15.58,6.31,20.11,4.67,28.60,3.70,21.41,0.34,28.04,0.27,2.42
Total non queer,4.35,22.52,2.69,16.22,2.91,14.33,5.76,19.23,5.48,28.25,4.72,22.64,0.21,25.15,0.12,1.35


In [548]:
sent_neo, sent_neut, sent_bin, hurt_neo, hurt_neut, hurt_bin, persp_neo, persp_neut, persp_bin, tot_neo, tot_neut, tot_bin = [], [], [], [], [], [], [], [], [], [], [], []
#scorro i modelli
for m in MODELS:
    eval = Evaluators(m)
    #prendo i valori di quel modello
    eval_sent = eval.sentiment_analysis_graph()
    eval_hurt = eval.hurtlex_graph()
    eval_pers = eval.perspective_graph()
    #metto il valore di quel modello nelle righe
    sent_neo.append(truncate_float(eval_sent['Neo'],2))
    sent_neut.append(truncate_float(eval_sent['Neutral'],2))
    sent_bin.append(truncate_float(eval_sent['Binary'],2))
    hurt_neo.append(truncate_float  (eval_hurt['Neo'],2))
    hurt_neut.append(truncate_float (eval_hurt['Neutral'],2))
    hurt_bin.append(truncate_float  (eval_hurt['Binary'],2))
    persp_neo.append(truncate_float  (eval_pers['Neo'],2))
    persp_neut.append(truncate_float (eval_pers['Neutral'],2))
    persp_bin.append(truncate_float  (eval_pers['Binary'],2))
    tot_neo.append(truncate_float  (total_score(eval_sent['Neo'], eval_hurt['Neo'], eval_pers['Neo']),2))
    tot_neut.append(truncate_float  (total_score(eval_sent['Neutral'], eval_hurt['Neutral'], eval_pers['Neutral']),2)) 
    tot_bin.append(truncate_float  (total_score(eval_sent['Binary'], eval_hurt['Binary'], eval_pers['Binary']),2))
df = pd.DataFrame(columns=MODELS_NAMES)
df.loc['Sentiment Neo'] = sent_neo
df.loc['Sentiment Neutral'] = sent_neut
df.loc['Sentiment Binary'] = sent_bin
df.loc['Hurtlex Neo'] = hurt_neo
df.loc['Hurtlex Neutral'] = hurt_neut
df.loc['Hurtlex Binary'] = hurt_bin
df.loc['Perspective Neo'] = persp_neo
df.loc['Perspective Neutral'] = persp_neut
df.loc['Perspective Binary'] = persp_bin
df.loc['Total Neo'] = tot_neo
df.loc['Total Neutral'] = tot_neut
df.loc['Total Binary'] = tot_bin
display(df)
df.to_csv(RESULTS_PATH+'total_score_pronouns.csv', sep=';', index=False)

Reading Perspective: 100%|██████████| 8268/8268 [00:00<00:00, 25262.31sentences/s]


,ALBERTbase[1],ALBERTbase[5],ALBERTlarge[1],ALBERTlarge[5],BERTbase[1],BERTbase[5],BERTlarge[1],BERTlarge[5],RoBERTabase[1],RoBERTabase[5],RoBERTalarge[1],RoBERTalarge[5],BERTweetbase[1],BERTweetbase[5],BERTweetlarge[1],BERTweetlarge[5]
Sentiment Neo,1.00,1.00,2.00,2.00,1.00,1.00,1.00,1.00,1.00,0.00,2.00,1.00,0.00,0.00,0.0,0.00
Sentiment Neutral,1.00,2.00,2.00,2.00,0.00,1.00,1.00,1.00,1.00,0.00,1.00,1.00,0.00,0.00,0.0,0.00
Sentiment Binary,1.00,1.00,3.00,2.00,2.00,1.00,2.00,1.00,1.00,0.00,2.00,2.00,0.00,0.00,0.0,0.00
Hurtlex Neo,2.10,7.69,1.66,7.61,0.57,9.14,0.87,9.14,2.02,13.49,1.88,12.98,0.00,85.41,0.0,0.43
Hurtlex Neutral,1.88,7.54,0.94,8.49,4.71,8.49,0.00,7.54,3.77,16.98,0.94,7.54,0.00,86.79,0.0,0.00
Hurtlex Binary,2.35,9.43,0.94,8.01,0.00,8.96,1.41,8.01,0.94,10.84,2.83,12.73,0.00,87.73,0.0,0.00
Perspective Neo,0.72,9.21,2.52,10.15,1.59,10.30,2.52,19.01,6.38,21.55,1.30,8.99,0.50,3.04,0.0,0.43
Perspective Neutral,1.88,5.66,0.94,5.66,0.00,6.60,0.00,9.43,5.66,22.64,0.00,2.83,0.00,0.00,0.0,0.00
Perspective Binary,3.30,16.03,0.47,11.32,0.00,9.90,2.83,15.09,4.24,20.75,0.00,0.94,0.00,0.94,0.0,0.00
Total Neo,1.60,6.30,2.73,7.25,1.39,7.14,1.80,10.05,3.47,11.68,2.39,7.98,0.16,29.48,0.0,0.28
